# GBM woe


## Adatok betöltése

In [230]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_bc050f45180b4c1bb8886542b85a43fe = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6akYbYHItNWmuiZPpgS8f0DnZNTVPSN3hYgF_EVLfHhd',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_bc050f45180b4c1bb8886542b85a43fe.get_object(Bucket='szakdolgozat-donotdelete-pr-nxru0g4tgksnkv',Key='WOE_train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

train_input = pd.read_csv(body, sep=';',decimal=',')
train_input.head(10)


,DPD90_12m,customer_type,product_type,onero,age_of_asset (honap),sales_channel,Nace risk tier
0,0,-0.029802,-0.180531,0.010955,0.603264,0.232891,0.585450
1,0,-0.029802,-0.180531,-0.210588,-1.386171,-0.883030,-0.893508
2,0,-0.029802,-0.180531,-0.210588,-0.365448,0.232891,-0.224232
3,0,-0.029802,-0.180531,0.728017,0.603264,0.232891,-0.224232
4,0,-0.029802,-0.180531,-0.210588,0.603264,0.232891,-0.224232
5,0,-0.029802,-0.180531,-0.210588,-0.365448,-0.883030,0.585450
6,0,0.276928,-0.180531,0.728017,0.603264,0.232891,0.585450
7,0,-0.029802,-0.180531,0.010955,0.603264,0.232891,0.585450
8,0,-0.029802,-0.180531,-0.210588,-0.365448,0.232891,-0.224232
9,0,-0.029802,-0.180531,-0.210588,-0.365448,0.232891,0.585450


In [231]:

body = client_bc050f45180b4c1bb8886542b85a43fe.get_object(Bucket='szakdolgozat-donotdelete-pr-nxru0g4tgksnkv',Key='WOE_test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

test_input = pd.read_csv(body, sep=';',decimal=',')
test_input.head()


,DPD90_12m,customer_type,product_type,onero,age_of_asset (honap),sales_channel,Nace risk tier
0,0,-0.029802,-0.180531,0.010955,0.603264,0.232891,0.585450
1,0,-0.029802,-0.180531,-0.210588,-1.386171,-0.883030,0.585450
2,0,-0.029802,-0.180531,0.728017,0.603264,0.232891,0.585450
3,0,-0.029802,-0.180531,0.728017,0.603264,0.232891,0.585450
4,0,-0.029802,-0.180531,-0.210588,0.603264,0.232891,-0.893508


In [232]:
train_input.shape

(7339, 7)

In [233]:
test_input.shape

(3083, 7)

## Adatok előkészítése

In [234]:
import numpy as np

In [235]:
X_train = np.asarray(train_input[['customer_type', 'product_type', 'onero', 'age_of_asset (honap)','sales_channel', 'Nace risk tier']])
X_train[0:1]

array([[0.01095466, 0.6032641 , 0.23289082, 0.58545026]])

In [236]:
X_test = np.asarray(test_input[['customer_type', 'product_type', 'onero', 'age_of_asset (honap)','sales_channel', 'Nace risk tier']])
X_test[0:1]

array([[0.01095466, 0.6032641 , 0.23289082, 0.58545026]])

In [237]:
y_train = np.asarray(train_input['DPD90_12m'])
y_train [0:5]

array([0, 0, 0, 0, 0])

In [238]:
y_test = np.asarray(test_input['DPD90_12m'])
y_test [0:5]

array([0, 0, 0, 0, 0])

## Modellezés

In [239]:
from sklearn.ensemble import GradientBoostingClassifier
LR = GradientBoostingClassifier(max_depth = 2, min_samples_split=0.05, n_estimators=100, learning_rate=0.1).fit(X_train,y_train)
LR

GradientBoostingClassifier(max_depth=2, min_samples_split=0.05)

In [240]:
yhat_test = LR.predict_proba(X_test)
yhat_test

array([[0.9949269 , 0.0050731 ],
       [0.92076778, 0.07923222],
       [0.99741778, 0.00258222],
       ...,
       [0.909774  , 0.090226  ],
       [0.82281808, 0.17718192],
       [0.99217706, 0.00782294]])

In [241]:
yhat_train = LR.predict_proba(X_train)
yhat_train

array([[0.9949269 , 0.0050731 ],
       [0.81961005, 0.18038995],
       [0.96382092, 0.03617908],
       ...,
       [0.9949269 , 0.0050731 ],
       [0.99741778, 0.00258222],
       [0.94824948, 0.05175052]])

In [242]:
tech, y_predicted_train = zip(*yhat_train)
y_predicted_train[0:5]

(0.005073098063715399,
 0.1803899470129446,
 0.03617907884529367,
 0.008716165752599713,
 0.020878127172955084)

In [243]:
tech, y_predicted_test = zip(*yhat_test)
y_predicted_test[0:5]

(0.005073098063715399,
 0.07923222050093906,
 0.002582222313861196,
 0.002582222313861196,
 0.04891782863411092)

## Modell ereje

In [244]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(y_train, y_predicted_train)
gini=2*roc-1
gini

0.551613060130639

In [245]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(y_test, y_predicted_test)
gini=2*roc-1
gini

0.5368613422505637